In [ ]:
import os, pickle, random
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, f1_score, accuracy_score
import matplotlib.pyplot as plt
from scipy.stats import mode
from transformers import DebertaV2Tokenizer, DebertaV2ForSequenceClassification, get_cosine_schedule_with_warmup
import torch
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import GradScaler, autocast

# -------------------- CONFIG --------------------
SEED = 42
BATCH_SIZE = 16
VAL_BATCH_SIZE = 32
MAX_LEN = 128
EPOCHS = 8
PATIENCE = 3
LR = 2e-5
MODEL_NAME = "microsoft/deberta-v3-small"
BASE_DIR = "/content/drive/MyDrive/FIRE"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------- UTILS --------------------
def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed()

# -------------------- DATASET --------------------
class CryptoDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self): return len(self.texts)
    def __getitem__(self, idx):
        return {
            "text": self.texts[idx],
            "label": self.labels[idx]
        }

def collate_fn(batch):
    texts = [x["text"] for x in batch]
    labels = torch.tensor([x["label"] for x in batch], dtype=torch.long)
    encoding = tokenizer(texts, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt")
    return {
        "input_ids": encoding["input_ids"],
        "attention_mask": encoding["attention_mask"],
        "labels": labels
    }

# -------------------- LOSSES --------------------
class FocalLoss(torch.nn.Module):
    def __init__(self, alpha=None, gamma=2.0, label_smoothing=0.1):
        super().__init__()
        self.alpha = torch.tensor(alpha).to(device) if alpha else None
        self.gamma = gamma
        self.smoothing = label_smoothing
    def forward(self, logits, target):
        ce = torch.nn.functional.cross_entropy(logits, target, reduction='none', label_smoothing=self.smoothing)
        pt = torch.exp(-ce)
        focal = (1 - pt) ** self.gamma * ce
        if self.alpha is not None:
            focal = self.alpha[target] * focal
        return focal.mean()

def supervised_contrastive_loss(embeddings, labels, temperature=0.3):
    norm = torch.nn.functional.normalize(embeddings, dim=1)
    sim = torch.matmul(norm, norm.T) / temperature
    labels = labels.view(-1, 1)
    mask = torch.eq(labels, labels.T).float().to(device)
    logits_mask = 1 - torch.eye(mask.size(0)).to(device)
    mask *= logits_mask
    exp_logits = torch.exp(sim) * logits_mask
    log_prob = sim - torch.log(exp_logits.sum(1, keepdim=True) + 1e-12)
    return -(mask * log_prob).sum(1).div(mask.sum(1) + 1e-12).mean()

# -------------------- TRAIN FUNCTION --------------------
def train_level3_model(
    train_loader, val_loader, save_path, y_train_labels,
    num_labels=4, contrastive_weight=0.3, level_name="level3_fold", use_amp=True
):
    model = DebertaV2ForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels, output_hidden_states=True).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
    total_steps = len(train_loader) * EPOCHS
    scheduler = get_cosine_schedule_with_warmup(optimizer, 0.1 * total_steps, total_steps)
    scaler = GradScaler(enabled=use_amp)

    class_counts = np.bincount(y_train_labels)
    class_weights = 1.0 / (np.log(1.01 + class_counts))
    class_weights = class_weights / class_weights.sum()
    focal_loss = FocalLoss(alpha=class_weights.tolist(), gamma=2.0, label_smoothing=0.1)

    best_f1, patience_counter = -1, 0
    train_loss_hist, val_f1_hist, train_f1_hist, train_acc_hist, val_acc_hist = [], [], [], [], []
    log_file_path = save_path.replace(".pth", "_log.txt")
    with open(log_file_path, "w") as log_file:
        for epoch in range(EPOCHS):
            model.train(); total_loss = 0; preds, targets = [], []
            for batch in tqdm(train_loader, desc=f"[{level_name}] Epoch {epoch+1}"):
                batch = {k: v.to(device) for k, v in batch.items()}
                optimizer.zero_grad()
                with autocast(enabled=use_amp):
                    out = model(**batch)
                    logits = out.logits
                    cls_emb = out.hidden_states[-1][:, 0]
                    loss = focal_loss(logits, batch["labels"])
                    if batch["labels"].unique().numel() > 1:
                        loss += contrastive_weight * supervised_contrastive_loss(cls_emb, batch["labels"])
                scaler.scale(loss).backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
                scheduler.step()
                total_loss += loss.item()
                preds += logits.argmax(dim=-1).cpu().tolist()
                targets += batch["labels"].cpu().tolist()

            train_loss_hist.append(total_loss)
            train_acc = accuracy_score(targets, preds)
            train_f1 = f1_score(targets, preds, average="weighted")
            train_f1_hist.append(train_f1)
            train_acc_hist.append(train_acc)

            model.eval(); val_preds, val_targets = [], []
            with torch.no_grad():
                for batch in val_loader:
                    batch = {k: v.to(device) for k, v in batch.items()}
                    with autocast(enabled=use_amp):
                        logits = model(**batch).logits
                    val_preds += logits.argmax(dim=-1).cpu().tolist()
                    val_targets += batch["labels"].cpu().tolist()
            val_f1 = f1_score(val_targets, val_preds, average="weighted")
            val_acc = accuracy_score(val_targets, val_preds)
            val_f1_hist.append(val_f1)
            val_acc_hist.append(val_acc)

            line = f" Epoch {epoch+1}: Train Loss = {total_loss:.4f} | Train Acc = {train_acc:.4f} | Train F1 = {train_f1:.4f} | Val Acc = {val_acc:.4f} | Val F1 = {val_f1:.4f}"
            print(line)
            log_file.write(line + "\n")
            if val_f1 > best_f1:
                best_f1 = val_f1
                patience_counter = 0
                torch.save(model.state_dict(), save_path)
                print(f" Model saved at: {save_path}")
                log_file.write(f"Saved model: {save_path}\n")
            else:
                patience_counter += 1
                if patience_counter >= PATIENCE:
                    print(" Early stopping")
                    log_file.write("Early stopping\n")
                    break

    # Plot & log
    plot_path = save_path.replace(".pth", "_plot.png")
    plt.figure(figsize=(10, 6))
    plt.plot(train_loss_hist, label="Train Loss")
    plt.plot(train_f1_hist, label="Train F1")
    plt.plot(train_acc_hist, label="Train Acc")
    plt.plot(val_f1_hist, label="Val F1")
    plt.plot(val_acc_hist, label="Val Acc")
    plt.title(f"{level_name} Training")
    plt.xlabel("Epoch")
    plt.legend(), plt.grid(True)
    plt.savefig(plot_path); plt.close()
    return model

# -------------------- MAIN SCRIPT --------------------
if __name__ == "__main__":
    run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_dir = os.path.join(BASE_DIR, f"run_{run_id}")
    os.makedirs(run_dir, exist_ok=True)
    for sub in ["models", "logs", "plots", "encoders"]: os.makedirs(os.path.join(run_dir, sub), exist_ok=True)

    train_df = pd.read_csv(f"{BASE_DIR}/crypto_task1_train.csv")
    val_df = pd.read_csv(f"{BASE_DIR}/crypto_task1_val.csv")

    print(" Unique values in level_1 (train):", train_df["level_1"].unique())
    print(" Unique values in level_2 (train):", train_df["level_2"].unique())

    train_l3_df = train_df[(train_df["level_1"] == 2) & (train_df["level_2"] == 0)].copy().reset_index(drop=True)
    val_l3_df = val_df[(val_df["level_1"] == 2) & (val_df["level_2"] == 0)].copy().reset_index(drop=True)

    if train_l3_df.empty:
        print(" No NEUTRAL samples found in level_2 under SUBJECTIVE. Skipping Level 3 training.")
    else:
        le3 = LabelEncoder()
        train_l3_df["level_3_enc"] = le3.fit_transform(train_l3_df["level_3"])
        val_l3_df["level_3_enc"] = le3.transform(val_l3_df["level_3"])
        pickle.dump(le3, open(f"{run_dir}/encoders/label_encoder_level_3.pkl", "wb"))

        tokenizer = DebertaV2Tokenizer.from_pretrained(MODEL_NAME)

        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
        level3_preds, level3_labels = [], []

        for fold, (tr_idx, va_idx) in enumerate(skf.split(train_l3_df, train_l3_df["level_3_enc"])):
            print(f"\n Fold {fold+1}/5")
            tr_df = train_l3_df.loc[tr_idx].reset_index(drop=True)
            va_df = train_l3_df.loc[va_idx].reset_index(drop=True)
            tr_loader = DataLoader(CryptoDataset(tr_df["text"], tr_df["level_3_enc"], tokenizer), batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
            va_loader = DataLoader(CryptoDataset(va_df["text"], va_df["level_3_enc"], tokenizer), batch_size=VAL_BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
            save_path = os.path.join(run_dir, "models", f"level3_fold{fold+1}.pth")

            model = train_level3_model(
                tr_loader, va_loader, save_path,
                y_train_labels=tr_df["level_3_enc"].values,
                level_name=f"level3_fold{fold+1}"
            )

            model.eval(); preds, targets = [], []
            with torch.no_grad():
                for batch in va_loader:
                    batch = {k: v.to(device) for k, v in batch.items()}
                    logits = model(**batch).logits
                    preds += logits.argmax(dim=-1).cpu().tolist()
                    targets += batch["labels"].cpu().tolist()
            level3_preds.append(preds)
            level3_labels.append(targets)

        pred_matrix = np.array(level3_preds)
        majority = mode(pred_matrix, axis=0).mode[0]
        true_labels = np.array(level3_labels[0])
        acc = accuracy_score(true_labels, majority)
        f1 = f1_score(true_labels, majority, average="weighted")
        print(f"\n Level 3 Ensemble Accuracy: {acc:.4f} | F1: {f1:.4f}")


🔍 Unique values in level_1 (train): [2 0 1]
🔍 Unique values in level_2 (train): [ 2.  0. nan  1.]

🔁 Fold 1/5


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-3-692409783.py:95: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=use_amp)
[level3_fold1] Epoch 1:   0%|          | 0/293 [00:00<?, ?it/s]/tmp/ipython-input-3-692409783.py:111: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=use_amp):
[level3_fold1] Epoch 1: 100%|██████████| 293/293 [00:51<00:00,  5.72it/s]
/tmp/ipython-input-3-692409783.py:137: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please u

📊 Epoch 1: Train Loss = 270.5017 | Train Acc = 0.4150 | Train F1 = 0.3758 | Val Acc = 0.5444 | Val F1 = 0.3839
💾 Model saved at: /content/drive/MyDrive/FIRE/run_20250629_121221/models/level3_fold1.pth


[level3_fold1] Epoch 2:   0%|          | 0/293 [00:00<?, ?it/s]/tmp/ipython-input-3-692409783.py:111: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=use_amp):
[level3_fold1] Epoch 2: 100%|██████████| 293/293 [00:43<00:00,  6.77it/s]
/tmp/ipython-input-3-692409783.py:137: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=use_amp):


📊 Epoch 2: Train Loss = 265.6484 | Train Acc = 0.5640 | Train F1 = 0.4558 | Val Acc = 0.7222 | Val F1 = 0.6716
💾 Model saved at: /content/drive/MyDrive/FIRE/run_20250629_121221/models/level3_fold1.pth


...

[level3_fold5] Epoch 7:   0%|          | 0/293 [00:00<?, ?it/s]/tmp/ipython-input-3-692409783.py:111: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=use_amp):
[level3_fold5] Epoch 7: 100%|██████████| 293/293 [00:45<00:00,  6.44it/s]
/tmp/ipython-input-3-692409783.py:137: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=use_amp):


📊 Epoch 7: Train Loss = 211.0344 | Train Acc = 0.8369 | Train F1 = 0.8227 | Val Acc = 0.8221 | Val F1 = 0.8112
💾 Model saved at: /content/drive/MyDrive/FIRE/run_20250629_121221/models/level3_fold5.pth


[level3_fold5] Epoch 8:   0%|          | 0/293 [00:00<?, ?it/s]/tmp/ipython-input-3-692409783.py:111: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=use_amp):
[level3_fold5] Epoch 8: 100%|██████████| 293/293 [00:45<00:00,  6.41it/s]
/tmp/ipython-input-3-692409783.py:137: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=use_amp):


📊 Epoch 8: Train Loss = 211.2309 | Train Acc = 0.8452 | Train F1 = 0.8358 | Val Acc = 0.8221 | Val F1 = 0.8131
💾 Model saved at: /content/drive/MyDrive/FIRE/run_20250629_121221/models/level3_fold5.pth


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (5,) + inhomogeneous part.